In [ ]:
### Install required libraries

!pip install --upgrade langchain==0.3.18 \
langchain-groq \
mem0ai \
pyautogen \
vecs \
supabase \
langmem==0.0.29 \
langgraph==0.6.4 \
langchain-google-genai \
langchain-community \
sentence-transformers


  Using cached langchain_groq-1.0.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_google_genai-3.0.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_google_genai-3.0.0-py3-none-any.whl.metadata (7.1 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.4-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.3.30-py3-none-any.whl.metadata (3.0 kB)
 

In [ ]:
### Imports

'''
ChatGroq is your LLM client.

entrypoint decorates your async function as a LangGraph “node” with state/store access.

InMemoryStore is the vector store (in-RAM).

create_memory_store_manager builds a pipeline that extracts “memories” from conversations and writes them to your store.

userdata fetches secrets you saved in Colab (e.g., GROQ_API_KEY).

SentenceTransformer loads your embedding model.
'''


# from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from langgraph.func import entrypoint
from langgraph.store.memory import InMemoryStore
from langmem import ReflectionExecutor, create_memory_store_manager
from google.colab import userdata
import asyncio
import time
import json
import uuid

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
#### Loads Jina v3 embeddings (1024-dim).

embedding_model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
#### The single encode("hello world") line warms up the model so the first real call is fast (and forces HF to download weights).

embedding_model.encode("hello world")

array([ 0.09548029, -0.09130365,  0.13924742, ...,  0.02013803,
       -0.01206389, -0.00295036], dtype=float32)

In [ ]:
#### embeddings turn text into numbers so you can search past info by meaning,

def embed(texts: list[str]) -> list[list[float]]:
    # normalize_embeddings=True is typical for cosine similarity
    return embedding_model.encode(texts, normalize_embeddings=True).tolist()

In [ ]:
''''
Creates an in-RAM vector index with the right dimension and your custom embedding function.

This is where preferences/memories are stored and later queried.

'''


store = InMemoryStore(
    index={
        "dims": 1024,     # Google embedding output dim (typically 768)
        "embed": embed,
    }
)




In [ ]:
#### Instantiates a Groq chat model with your API key.

llm = ChatGroq(model = "openai/gpt-oss-20b" , api_key = userdata.get('GROQ_API_KEY'))

In [ ]:
# Memory manager (automatic memory extraction)

'''
Builds a LangMem pipeline that reads your conversation and writes relevant facts (e.g., “Vishnu likes dairy”) into the vector store under a namespace.

Instead of you manually deciding what to save, this step auto-extracts and stores key facts.

'''

memory_manager = create_memory_store_manager(
    llm,
    namespace=("food_preferences",), )



In [ ]:
# Global variable to control verbosity - print helpful diagnostics (retrieval results, timings, new memories).
verbose_mode = True

In [ ]:
#### The agent function (LangGraph entrypoint)



@entrypoint(store=store)
async def recipe_assistant(message: str):
    """
    A recipe assistant that remembers food preferences.
    """
    global verbose_mode

    if verbose_mode:
        print("\n" + "-" * 60)
        print("🔄 MEMORY SYSTEM INTERNALS:")
        print("-" * 60)

    # Step 1: Vector search for relevant memories
    if verbose_mode:
        print("\n🔍 SEARCHING VECTOR STORE")
        print(f"  Query: '{message}'")
        print(f"  Namespace: ('food_preferences',)")
        print(f"  Limit: 3")

    #### ---- First, check if there are any relevant food preferences in memory. Finds the most relevant memories to the new message.
    relevant_memories = store.search(("food_preferences",), query=message, limit=3)

    #### ---- Display embedding and similarity details for memory retrieval
    if verbose_mode:
        if relevant_memories:
            print("\n  Results:")
            for i, memory in enumerate(relevant_memories, 1):
                print(f"  {i}. Memory ID: {memory.key}")
                print(f"     Content: {memory.value['content']['content']}")
                if hasattr(memory, "score"):
                    print(f"     Relevance score: {memory.score}")
                print(f"     Created at: {memory.created_at}")
        else:
            print("\n  No relevant memories found")

    # Format memories as context for the LLM
    memory_context = ""
    if relevant_memories:
        memory_context = "Previous information about the user's food preferences:\n"
        for memory in relevant_memories:
            memory_context += f"- {memory.value['content']['content']}\n"

    # Step 2: Construct prompt with memory context
    if verbose_mode:
        print("\n📋 CONSTRUCTING AUGMENTED PROMPT")
        print(f"  Base message: '{message}'")
        if memory_context:
            print(f"  Prepending {len(relevant_memories)} memories as context")
        else:
            print("  No memory context to prepend")

    #### ---- Prepend the retrieved memories so the LLM can answer in context (
    prompt = f"{memory_context}\nUser message: {message}\n\nPlease provide a helpful response about recipes or food, using any relevant previous preferences if appropriate."

    if verbose_mode:
        print("\n  Final prompt to LLM:")
        print("  " + "-" * 40)
        for line in prompt.split("\n"):
            print(f"  {line}")
        print("  " + "-" * 40)

    # Step 3: Generate response from the LLM
    if verbose_mode:
        print("\n🤖 GENERATING LLM RESPONSE")


    #### ---- The assistant replies, benefiting from the prepended memory.
    start_time = time.time()
    response = llm.invoke(prompt)
    end_time = time.time()

    if verbose_mode:
        print(f"  Response time: {end_time - start_time:.2f} seconds")
        print(f"  Response length: {len(response.content)} characters")

    # Step 4: Memory extraction process
    if verbose_mode:
        print("\n🧠 MEMORY EXTRACTION PROCESS")
        print("  Packaging conversation for memory extraction:")
        print("  - User message")
        print("  - Assistant response")

    # Create conversation object to process
    conversation_id = str(uuid.uuid4())
    if verbose_mode:
        print(f"  Conversation ID: {conversation_id}")

    to_process = {"messages": [{"role": "user", "content": message}] + [response]}

    if verbose_mode:
        print("\n  Sending to memory manager for extraction...")

    # Get the count of memories before extraction
    before_count = len(list(store.search(("food_preferences",))))

    # Extract memories from the conversation
    memory_extraction_start = time.time()
    extraction_result = await memory_manager.ainvoke(to_process)
    memory_extraction_end = time.time()

    if verbose_mode:
        print(
            f"  Extraction time: {memory_extraction_end - memory_extraction_start:.2f} seconds"
        )

    # Step 5: Check for new memories
    after_count = len(list(store.search(("food_preferences",))))
    new_count = after_count - before_count

    if verbose_mode:
        print(f"\n💾 MEMORY STORAGE RESULTS")
        print(f"  Memories before: {before_count}")
        print(f"  Memories after: {after_count}")
        print(f"  New memories added: {new_count}")

        if new_count > 0:
            print("\n  New memories:")
            # Get all memories and sort by creation time to find newest
            all_memories = list(store.search(("food_preferences",)))
            all_memories.sort(key=lambda x: x.created_at, reverse=True)

            for i, memory in enumerate(all_memories[:new_count], 1):
                print(f"  {i}. ID: {memory.key}")
                print(f"     Content: {memory.value['content']['content']}")
                print(f"     Created at: {memory.created_at}")

        print("\n" + "-" * 60 + "\n")

    return response.content



In [ ]:
#### Interactive console (Colab cell)


'''
Simple REPL for testing: you type; it runs the whole pipeline; you see the result.

In Colab, run it with await interactive_console() (not asyncio.run(...)).

'''

async def interactive_console():
    """
    Interactive console interface for the recipe assistant.
    """
    global verbose_mode

    print("\n" + "=" * 70)
    print("🍳 Welcome to the Interactive Recipe Assistant with Memory Visualization 🍳")
    print("This assistant remembers your food preferences over time.")
    print("Commands:")
    print("  'exit' or 'quit': End the conversation")
    print("  'memories': View all stored memories")
    print("  'verbose on/off': Toggle detailed memory system visibility")
    print("=" * 70 + "\n")

    while True:
        # Get user input
        user_input = input("YOU: ")

        # Handle special commands
        if user_input.lower() in ["exit", "quit"]:
            print("\nThank you for using the Recipe Assistant. Goodbye!")
            break
        elif user_input.lower() == "memories":
            print("\n🧠 ALL STORED MEMORIES:")
            memories = list(store.search(("food_preferences",)))
            if memories:
                # Sort by creation time, newest first
                memories.sort(key=lambda x: x.created_at, reverse=True)
                for i, memory in enumerate(memories, 1):
                    print(f"{i}. ID: {memory.key}")
                    print(f"   Content: {memory.value['content']['content']}")
                    print(f"   Created: {memory.created_at}")
                    print()
            else:
                print("No memories stored yet.")
            print()
            continue
        elif user_input.lower() == "verbose on":
            verbose_mode = True
            print("Verbose mode enabled - showing memory system details")
            continue
        elif user_input.lower() == "verbose off":
            verbose_mode = False
            print("Verbose mode disabled - hiding memory system details")
            continue

        # Process normal input through the assistant
        response = await recipe_assistant.ainvoke(user_input)
        print(f"\nASSISTANT: {response}\n")

In [ ]:
await interactive_console()


🍳 Welcome to the Interactive Recipe Assistant with Memory Visualization 🍳
This assistant remembers your food preferences over time.
Commands:
  'exit' or 'quit': End the conversation
  'memories': View all stored memories
  'verbose on/off': Toggle detailed memory system visibility

YOU: My name is karan and I love eggs

------------------------------------------------------------
🔄 MEMORY SYSTEM INTERNALS:
------------------------------------------------------------

🔍 SEARCHING VECTOR STORE
  Query: 'My name is karan and I love eggs'
  Namespace: ('food_preferences',)
  Limit: 3

  No relevant memories found

📋 CONSTRUCTING AUGMENTED PROMPT
  Base message: 'My name is karan and I love eggs'
  No memory context to prepend

  Final prompt to LLM:
  ----------------------------------------
  
  User message: My name is karan and I love eggs
  
  Please provide a helpful response about recipes or food, using any relevant previous preferences if appropriate.
  ---------------------------